# Numerical Optimization using JuMP


# JuMP

[JuMP](https://github.com/JuliaOpt/JuMP.jl) is a Julia package that provides a modeling language for general optimization problems.

## Optimization in Julia

Julia's optimization packages have become popular due to their ease of use, and variety of interfaces to mature solvers.  The main optimization functionality in Julia is provided by [JuliaOpt](http://www.juliaopt.org/).

The two high-level interfaces that you are most likely to use are
* [JuMP](https://github.com/JuliaOpt/JuMP.jl) - a modeling language for all sorts of optimization problems
* [Convex.jl](https://github.com/JuliaOpt/Convex.jl) - a package for disciplined convex programming (like [CVX](http://cvxr.com/cvx/))

There is a mid-level interface as well, [MathProgBase.jl](https://github.com/JuliaOpt/MathProgBase.jl).

The powerful thing about all of the above is that it is largely **solver independent**. This means you can forumlate the optimization problem with these packages, and then choose from a variety of solvers to use under the hood.  This is just like other modeling languages like AMPL - the reason why Julia's optimization packages have become popular is that they are generally easier to use than older modeling languages.

## Solvers

Today is more about turning your optimization models into something that can run on a computer via JuMP, but you still need a solver to actually solve the problem for you under the hood.  [JuliaOpt](http://www.juliaopt.org/) has a list of solvers that can be called from the high level interfaces (there are currently 20).  There are many open-source options available, but there are also interfaces to some of the big commercial solvers such as [Gurobi](http://www.gurobi.com/), [Mosek](https://www.mosek.com/), [Knitro](https://www.artelys.com/en/optimization-tools/knitro), etc. Many of these commercial solvers offer free academic/student licences, and if you are trying to solve large optimization problems they may be worth looking at.

In [1]:
# If you have not already installed JuMP and a solver
Pkg.add("JuMP")
Pkg.add("Clp")   # solver
Pkg.add("Ipopt") # solver

INFO: Package JuMP is already installed
INFO: METADATA is out-of-date — you may not have the latest version of JuMP
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Package Clp is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Clp
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [4]:
using JuMP
using Clp

INFO: Precompiling module Clp.


# Optimization Problems

The hardest part of solving an optimization problem is often formulating the problem itself.  Optimization problems generally take the form

$$
\text{minimize}_x~f(x)\\
\text{subject to}~c_e(x) = 0; c_i(x) \ge 0
$$

where $f(x)$ is some function, $c_e(x)$ encodes equality constraints, and $c_i(x)$ encodes inequality constraints.

# Linear Programming

[Linear Programs (LPs)](https://en.wikipedia.org/wiki/Linear_programming) are optimization problems for which both the objective and constraint functions are linear.

$$
\text{minimize}_{x\in \mathbb{R}^n}~c^T x\\
\text{subject to}~Ax = a; Bx \ge b
$$

One place LPs arise are in maximizing profit.

Here's a very simple LP we can easily solve by hand:

$$
\text{minimize}~x+y\\
\text{subject to:}\\
\qquad x\ge 0\\
\qquad y \ge 0\\
\qquad x+y \le 1
$$

In [26]:
using Cbc

In [22]:
# create a model
# m = Model(solver=ClpSolver())
m = Model(solver=GurobiSolver())
# add variables
@variable(m, x >= 0)
@variable(m, y >= 0)
# add additional constraint
@constraint(m, x+y <= 1)
# define objective
@objective(m, Min, x+y)
# prints problem we've defined
m

Minimization problem with:
 * 1 linear constraint
 * 2 variables
Solver is Gurobi

In [23]:
# solve the problem
solve(m)
# print objective value, and the values of x,y
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00
Optimal objective: 0.0. x = 0.0 y = 0.0


# Exercise

1. Install another solver that will solve a Linear Program (see [JuliaOpt](http://www.juliaopt.org/) for options - look at the first column), and solve the above example using the new solver.

# Quadratic Programming

[Quadratic Programs (QPs)](https://en.wikipedia.org/wiki/Quadratic_programming) allow the objective function $f(x)$ to be quadratic.  The optimization problems have the form

$$
\text{minimize}_{x\in \mathbb{R}^n}~\tfrac{1}{2}x^T Q x+c^T x\\
\text{subject to}~Ax \le b
$$

Note that in the case that $Q$ is SPD, that the problem is convex, but this is generally not the case.

In [29]:
using Ipopt

INFO: Precompiling module Ipopt.


In [33]:
# create a model
m = Model(solver=IpoptSolver())
# add variables
@variable(m, x >= 0)
@variable(m, y >= 0)
# add additional constraint
@constraint(m, x+y <= 1)
# define objective
@objective(m, Min, (x-0.5)^2 + y)
# prints problem we've defined
m

Minimization problem with:
 * 1 linear constraint
 * 2 variables
Solver is Ipopt

In [34]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

# Second-order Cone Constraints

http://www.juliaopt.org/JuMP.jl/0.18/refmodel.html#second-order-cone-constraints

# Integer Programming

[Integer Programs (IPs)](https://en.wikipedia.org/wiki/Integer_programming) add integer constraints to the optimization variables.  This makes sense when variables come in unit quantities (e.g. people).

In [48]:
m = Model(solver=CbcSolver())
@variable(m, x >= 0, Int) # Int keyword says that x should be an integer
@variable(m, y >= 0, Int)

@objective(m, Max, x + y)
@constraint(m, 50x + 24y <= 2400)
@constraint(m, 30x + 33y <= 2100)
m

Maximization problem with:
 * 2 linear constraints
 * 2 variables: 2 integer
Solver is CbcMathProg

In [49]:
solve(m)
println("Optimal objective: ",getobjectivevalue(m), 
	". x = ", getvalue(x), " y = ", getvalue(y))

Optimal objective: 66.0. x = 31.0 y = 35.0


# Mixed Integer Programming

Mixed Integer Programs (MIPs) have some variables that are constrained to be integers, and some that are not.

callbacks:

http://www.juliaopt.org/JuMP.jl/0.18/callbacks.html

# Non-linear Programming

http://www.juliaopt.org/JuMP.jl/0.18/nlp.html